# 04_summarize_complaints_by_CPA.ipynb

## Purpose
Summarize the cleaned and spatially joined complaint points to Community Planning Areas (CPAs).

## Expected Input
`COMPLAINTS_DTL_WITH_CPA` is produced in Notebook 3 and contains **only matched complaint records** (i.e., complaints that successfully intersected a CPA polygon).  
Unmatched records (Join_Count = 0 / null) have already been filtered into a separate feature class (e.g., `COMPLAINTS_DTL_WITH_CPA_ERRORS`) and are **not included** in this summary.

## Output
A CPA polygon feature class with ~61 records (one per CPA) containing:
- all CPA fields from `CPA_FC_PRJ`
- `number_of_complaints` = count of complaints within each CPA


In [5]:
import os
import arcpy

arcpy.env.overwriteOutput = True


PROJECT_ROOT = r"C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone"

WORK_DIR = os.path.join(PROJECT_ROOT, "data_working")
GDB_PATH = os.path.join(WORK_DIR, "GetItDoneAnalysis.gdb")

# Inputs
CPA_FC_PRJ = os.path.join(GDB_PATH, "cpa_prj")
COMPLAINTS_DTL_WITH_CPA_MATCHED = os.path.join(GDB_PATH, "gid_drainage_points_lastmonth_with_cpa")

# Output
CPA_SUMMARY_FC = os.path.join(GDB_PATH, "gid_drainage_by_cpa_lastmonth")

arcpy.env.workspace = GDB_PATH

for p in [CPA_FC_PRJ, COMPLAINTS_DTL_WITH_CPA_MATCHED]:
    if not arcpy.Exists(p):
        raise FileNotFoundError(f"Missing input: {p}")

print("CPA_FC_PRJ:", CPA_FC_PRJ)
print("COMPLAINTS_DTL_WITH_CPA_MATCHED:", COMPLAINTS_DTL_WITH_CPA_MATCHED)


CPA_FC_PRJ: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\GetItDoneAnalysis.gdb\cpa_prj
COMPLAINTS_DTL_WITH_CPA_MATCHED: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\GetItDoneAnalysis.gdb\gid_drainage_points_lastmonth_with_cpa


In [26]:

fieldmappings = arcpy.FieldMappings()

# Add all fields from inputs.
fieldmappings.addTable(CPA_FC_PRJ)
fieldmappings.addTable(COMPLAINTS_DTL_WITH_CPA_MATCHED)

# Name fields to keep. 
keepers = ["OBJECTID","Shape","Join_Count_1","TARGET_FID","cpcode","CPA_NAME","acreage","CPA_ID","AREA_SQMI"] 

# Remove all output fields you don't want.
for field in fieldmappings.fields:
    if field.name not in keepers:
        fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field.name))

if arcpy.Exists(CPA_SUMMARY_FC):
    arcpy.management.Delete(CPA_SUMMARY_FC)

# Spatial join: output matches CPA polygons (one row per CPA)
arcpy.analysis.SpatialJoin(
    target_features=CPA_FC_PRJ,            # polygons (≈61)
    join_features=COMPLAINTS_DTL_WITH_CPA_MATCHED, # matched points only
    out_feature_class=CPA_SUMMARY_FC,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="INTERSECT"
)

print("Created CPA summary FC:", CPA_SUMMARY_FC)
print("CPA summary count:", arcpy.management.GetCount(CPA_SUMMARY_FC)[0])

objectid
Join_Count
service_request_id
service_name
service_name_detail
public_description
date_requested
status
street_address
QA_STATUS
lat
lon
objectid_1
Created CPA summary FC: C:\Users\kris_\OneDrive - Kris Manske\Documents\Classes\BootcampGIS\Wildfire repositories on AWS\GetItDone\data_working\GetItDoneAnalysis.gdb\gid_drainage_by_cpa_lastmonth
CPA summary count: 61


In [16]:
# Confirm expected field exists and inspect basic totals
fields = [f.name for f in arcpy.ListFields(CPA_SUMMARY_FC)]
print("Has number_of_complaints:", "number_of_complaints" in fields)

# Sum of complaints across CPAs should equal count of input points (matched)
input_cnt = int(arcpy.management.GetCount(COMPLAINTS_DTL_WITH_CPA_MATCHED)[0])

total = 0
with arcpy.da.SearchCursor(CPA_SUMMARY_FC, ["number_of_complaints"]) as cur:
    for (n,) in cur:
        total += int(n) if n is not None else 0

print("Input matched points count:", input_cnt)
print("Sum of number_of_complaints across CPAs:", total)



Has number_of_complaints: True
Input matched points count: 3324
Sum of number_of_complaints across CPAs: 3324
